# BigARTM CLI


## Приложение `bigartm.exe`

Приложение `bigartm.exe` (в Unix — просто `bigartm`) — самостоятельный бинарик, работающий по следующему алгоритму:

1. Загрузить словарь, если он указан
2. Инициализировать модель
3. Если батчи еще не созданы — прочитать корпус в указанном формате и создать батчи
4. Сделать определенное число итераций Online-EM алгоритма по указанным батчам, в соответствии с настройками
5. При последнем проходе по корпусу, вывести в файл предсказанные тематические профили документов (вектора $\theta$)
6. Вывести модель/словарь в файлы

#### Самостоятельный бинарник

Бинарник не должен зависеть от динамически линкуемых библиотек. При переносе бинарника на другую машину он должен запускаться без дополнительных файлов и переменных окружения. Самостоятельность бинарика значительно упростит его использование на кластере для распределенного обучения модели и вывода тематических профилей документов. 

Для сборки самостоятельного бинарика `gcc` нужно передавать флаг `-static`. В `cmake` нужно прописать что-то такое:
```
SET(CMAKE_EXE_LINKER_FLAGS "-static")
```

## Ключи консольного приложения

- `--rand-seed <seed>`: инициализация генератора случайных чисел, гарантируется что приложение отрабатывает с одинаковым результатом при одинаковом значении seed. По-умолчанию выбирается по таймеру (случайно).

### Dictionary

Использование существующего словаря:
- `--use-dictionary <dictionary-file>`: использовать словарь, загруженный из файла в бинарном формате
- `--use-dictionary-bow <vocab-txt>`: использовать текстовый словарь в формате vocab.xxx.txt из UCI Bag-of-Words
- настройки динамического словаря (в случае если он меняется), мин частота, макс число токенов

Если не указана опция использования существующего словаря, то по-умолчанию создается новый словарь.

Создание нового словаря по данным (токены добавляются автоматически как в [примере](https://github.com/bigartm/bigartm/blob/master/src/python/examples/example14_initialize_topic_model.py)):
- `--dictionary-min-items <N/P>`: брать только слова, которые присутствуют не менее чем в N документах (если указано целое >= 1) или P% документов (если указано вещественное < 1)
- `--dictionary-max-items <N/P>`: ... не более чем в N документах
- `--dictionary-min-total-count <N>`: брать только слова, которые встретились не менее N раз во всей коллекции

При построении словаря учитываются опции `use-modalities / ignore-modalities` (из Learning Options): словарь по неиспользуемым модальностям не составляется.

### Corpus

- `--corpus-format [vw,bow,mm]`: формат в котором потупает корпус текстов
- `--read-corpus <file/url>`: источник, из которого поступает корпус в сыром виде: stdin / файл / каталог-с-файлами / сеть
- `--read-batches <batches-path>`: путь к батчам, использовать заранее подготовленные батчи

По умолчанию считается что подготовленные батчи отсутствуют и сырой корпус читается из stdin в формате vw.

### Model

- `--load-model <model-file>`: загрузить модель из файла

Если не указана опция загрузки модели, то модель инициализируется.

- `--initialize <scheme>`: создать новую модель, инициализировав ее одной из доступных схем 
- `--num-topics`: задать фиксированное число тем
- `--topics <topics-file>`: указать файл с названиями топиков

### Learning Options

Базовые ключи:
- `--passes <N>`: число проходов по корпусу, по умолчанию N=1

Параметры обучения:
- `--inner-iterations-count <N>`
- `--decay-weight <w>`
- `--apply-weight <w>`

Модальности:
- `--use-modalities <modalities>`: использовать только указанные модальности
- `--ignore-modalities <modalities>`: отключить указанные модальности (используется одно из двух: use-modalities либо ignore-modalities)
- `--modality-weights <weights>`: назначить веса различным модальностям
- `--modality-weights word:1,author:10.0,link:0.2`

Регуляризация:
- регуляризатор задается набором `(вес, тип, параметры, модальности, топики)`
- модальности указываются в формате `@name`
- топики указываются в формате `#topic`
- `--regularizer "0.1 smoothPhi"`
- `--regularizer "25.0 sparseTheta"`
- `--regularizer "10 decorrelation @word"`
- `--regularizer "0.1 specific(x,y) #topic1:0.1,topic2:2.0 @word:0.5,author:0.1"`


### Multicore Mode

- `--threads <N>`: число потоков в которых производить обучение, по умолчанию N=1

### Distributed Mode

*Пока не нужно реализовывать.*

### Output

- `--write-batches <batches-path>`: при первом чтении генерировать батчи и записывать по назначению
- `--write-model <model-file>`: по окончанию обучения записать модель в файл
- `--write-dictionary <dictionary-file>`: вывести словарь в файл
- `--write-predictions <predictions-file>`: вывести тематические профили документов с последнего прохода по коллекции, в формате CSV. Возможность вывода в stdout!
- `--write-model-readable <model-csv-file>`: по окончанию обучения записать модель в текстовый файл в формате CSV

- `--batch-size <N>`: число документов в одном батче

## Информационный вывод и логирование

Информационный вывод должен производиться исключительно в stderr, поскольку в stdout может подаваться результат — тематические профили документов.

Что выводить в stderr:
- число проходов
- число обработанных документов
- метрики качества
  - перплексия
  - разреженность по модальностям
- способы подсчета метрик: hold-out / [progressive-validation](http://hunch.net/~jl/projects/prediction_bounds/thesis/mathml/thesisse44.xml) 
- логарифмический вывод (как в vw)

## Форматы выходных файлов

## Примеры использования

На этих примерах нужно протестировать новое CLI, а в конечном счете – сделать подробное описание этих примеров в документации.

### 1. Эксперимент на датасетах UCI Bag-of-Words

Предположим, что мы загрузили [данные с UCI](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words) в каталог `data`.

Обучение ванильной модели PLSA
```bash
bigartm \
    --use-dictionary-bow data/vocab.enron.txt \
    --corpus-format bow \
    --read-corpus data/docword.enron.txt \
    --num-topics 50 \
    --passes 30 \
    --write-model enron_50topics.model \
    --write-predictions enron_50topics.theta.txt \
    --write-model-readable enron_50topics.phi.txt 
```

Инициализируем модель для корпуса `nytimes` несколькими итерациями без регуляризаторов, создадим 100 тем с префиксом `topic` (целевые темы) и 3 темы `noise` (шумовые):
```bash
bigartm \
    --use-dictionary-bow data/vocab.nytimes.txt \
    --corpus-format bow \
    --read-corpus data/docword.nytimes.txt \
    --batch-size 1000 \
    --topics "topic[100],noise[3]" \ # создает 103 темы: topic0,topic1,...,topic99,noise0,...,noise2
    --passes 5 \
    --write-model nytimes_100topics_3noise.init.model \
    --write-batches nytimes.batches \ # название каталога с батчами, который необходимо создать
    --write-dictionary nytimes.dict \ # словарь в бинарном формате
    --threads 8  # первая итерация происходит последовательно, поскольку нет батчей
                 # начиная со второй — батчи лежат в nytimes.batches, BigARTM начинает работать параллельно
```

Продолжим обучение модели. Теперь включаем регуляризаторы; шумовые темы будут сглаживаться, а целевые — разреживаться и декоррелироваться:
```bash
bigartm \
    --use-dictionary nytimes.dict \
    --read-batches nytimes.batches \
    --load-model nytimes_100topics_3noise.init.model \
    --regularizer "0.5 smoothPhi #noise*" \
    --regularizer "0.5 sparsePhi #topic*" \
    --regularizer "0.5 smoothTheta #noise*" \
    --regularizer "0.5 sparseTheta #topic*" \
    --regularizer "1000 decorrelationPhi #topic*" \
    --passes 50 \
    --decay-weight 0.9 \  # детальная настройка итераций
    --apply-weight 0.1 \
    --write-model nytimes_100topics_3noise.model \
    --write-predictions nytimes_100topics_3noise.theta.txt \
    --write-model-readable nytimes_100topics_3noise.phi.txt \
    --threads 32
```

### 2. Обучение модели мультиязычной Википедии и использование для разметки документов

По корпусу статей Википедии построим многоязычную тематическую модель, которую затем сможем использовать для категоризации произвольных страниц.

Для обучения тематической модели был подготовлен мультиязычный корпуc в построчном формате vw, его положили в партицированном виде в HDFS (как результат работы MapReduce задачи, к примеру):
```bash
hdfs -cat wikipedia_corpus/part_00001

  page_2342123 |en computer:2 device:3 mouse input |ru мышь:2 устройство компьютер |es ratón dispositivo:3 computadora:2
  page_5645623 |en crusade:4 heretics:2 jerusalem |fr croisades:3 hérétique:2
  ...
```

Инициализируем модель и словарь, корпус подается на вход через stdin:
```bash
hdfs -cat wikipedia_corpus/\* | bigartm \
    --dictionary-min-items 3 \   # брать слово если оно нашлось не менее чем в 3х документах
    --dictionary-max-items 0.3 \ #  ... не более чем в 30% документов
    --corpus-format vw \
    --read-corpus - \ # минус значит что читать из stdin (так принято в unix)
    --topics topic[1000] \
    --use-modalities en,ru,fr,es,de,it \
    --write-batches wikipedia.batches \
    --write-dictionary wikipedia.dict \
    --write-model wikipedia_1k_topics.initial.model
```

Теперь обучим хорошую модель, параллельно:
```bash
bigartm \
    --read-batches wikipedia.batches \
    --load-model wikipedia_1k_topics.initial.model \
    --passes 100 \
    --regularizer "0.5 sparsePhi #topic*" \
    --decay-weight 0.95 \
    --write-model wikipedia_1k_topics.model \
    --write-model-readable wikipedia_1k_topics.phi.txt \
    --threads 32
```

Ура, теперь можем использовать модель для разметки документов по темам:
```bash
echo "new_document |ru пхнглуи мглвнафх рльех вгахнагл фхтагн ктулху" | bigartm  \
    --load-model wikipedia_1k_topics.model \
    --corpus-format vw \
    --read-corpus - \     # прочитать корпус из stdin 
    --write-predictions - # записать предсказания в stdout
    
  new_document    0    0    0    0.3    0    0    0    ...    0    0.1    0
```
В таком виде BigARTM можно использовать для разметки произвольно большого набора документов при помощи MapReduce.

## Пакетирование

### Пакет для Debian

В идеале BigARTM пользователь должен не собирать из исходников, а устанавливать вот таким образом:
`apt-get install bigartm`

Что нужно при этом установить:
- SO: `/usr/lib/bigartm.so`
- CLI: `/usr/bin/bigartm`
- Python-пакет: `/usr/lib/python2.7/site-packages`

[Руководство по созданию deb-пакетов](http://ubuntuforums.org/showthread.php?t=910717)